In [ ]:
"""
TMD Library Demo - Demonstrates the main features of the TMD library
"""
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from tmd.processor import TMDProcessor
from tmd.processing import (
    crop_height_map, flip_height_map, rotate_height_map,
    threshold_height_map, extract_cross_section
)
from tmd.exporters.numpy import export_to_npy, export_to_npz, export_metadata_txt
from tmd.exporters.image import (
    convert_heightmap_to_displacement_map, convert_heightmap_to_normal_map,
    convert_heightmap_to_bump_map
)
from tmd.exporters.stl import convert_heightmap_to_stl


def create_sample_height_map():
    """Create a sample height map for demonstration purposes."""
    print("Creating sample height map...")
    x = np.linspace(-5, 5, 100)
    y = np.linspace(-5, 5, 100)
    X, Y = np.meshgrid(x, y)
    
    # Create a synthetic height map with some features
    Z = (
        np.sin(X) * np.cos(Y) +                 # Wave pattern
        np.exp(-(X**2 + Y**2) / 8) * 2 +        # Central peak
        np.random.normal(0, 0.1, X.shape)       # Add some noise
    )
    
    # Normalize to [0, 1] range
    Z = (Z - Z.min()) / (Z.max() - Z.min())
    
    return Z.astype(np.float32)


def generate_synthetic_tmd():
    """Generate a synthetic TMD file for demonstration purposes."""
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)
    
    tmd_path = os.path.join(output_dir, "sample.tmd")
    
    print(f"Generating synthetic TMD file: {tmd_path}")
    
    # Create the height map
    height_map = create_sample_height_map()
    
    # Write a basic TMD file
    with open(tmd_path, 'wb') as f:
        # Write header
        f.write(b'Binary TrueMap Data File v2.0\0')
        # Padding to reach offset 32
        f.write(b'\0' * (32 - 26))
        # Add a comment
        comment = b'TMD Demo Example File\0'
        f.write(comment)
        # Padding to reach offset 64
        f.write(b'\0' * (64 - 32 - len(comment)))
        
        # Add dimensions
        width, height = height_map.shape
        f.write(width.to_bytes(4, byteorder='little'))
        f.write(height.to_bytes(4, byteorder='little'))
        
        # Add spatial info (all floats)
        x_length = 10.0  # mm
        y_length = 10.0  # mm
        x_offset = 0.0   # mm
        y_offset = 0.0   # mm
        
        import struct
        f.write(struct.pack('<ffff', x_length, y_length, x_offset, y_offset))
        
        # Add height data
        f.write(height_map.tobytes())
    
    return tmd_path


def main():
    """Main function demonstrating TMD library features."""
    # Set up output directory
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)
    
    # 1. Generate a synthetic TMD file
    tmd_path = generate_synthetic_tmd()
    
    # 2. Process the TMD file
    print("\nProcessing TMD file...")
    processor = TMDProcessor(tmd_path)
    processor.set_debug(True)
    data = processor.process()
    
    if not data:
        print("Failed to process TMD file. Using synthetic height map directly.")
        height_map = create_sample_height_map()
        data = {
            'height_map': height_map,
            'width': height_map.shape[1],
            'height': height_map.shape[0],
            'x_length': 10.0,
            'y_length': 10.0,
            'x_offset': 0.0,
            'y_offset': 0.0,
            'comment': 'Synthetic height map',
            'header': 'Synthetic TMD'
        }
    
    # 3. Basic height map visualization
    height_map = data['height_map']
    plt.figure(figsize=(10, 8))
    plt.imshow(height_map, cmap='viridis')
    plt.colorbar(label='Height')
    plt.title('Raw Height Map')
    plt.savefig(os.path.join(output_dir, 'raw_height_map.png'))
    plt.close()
    
    # 4. Demonstrate processing operations
    print("\nDemonstrating processing operations...")
    
    # Crop a region of the height map
    rows, cols = height_map.shape
    crop_region = (rows//4, 3*rows//4, cols//4, 3*cols//4)  # Middle 50%
    cropped = crop_height_map(height_map, crop_region)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(cropped, cmap='viridis')
    plt.colorbar(label='Height')
    plt.title('Cropped Height Map')
    plt.savefig(os.path.join(output_dir, 'cropped_height_map.png'))
    plt.close()
    
    # Flip the height map
    flipped = flip_height_map(height_map, axis=0)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(flipped, cmap='viridis')
    plt.colorbar(label='Height')
    plt.title('Flipped Height Map (Horizontal)')
    plt.savefig(os.path.join(output_dir, 'flipped_height_map.png'))
    plt.close()
    
    # Rotate the height map
    rotated = rotate_height_map(height_map, angle=45)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(rotated, cmap='viridis')
    plt.colorbar(label='Height')
    plt.title('Rotated Height Map (45°)')
    plt.savefig(os.path.join(output_dir, 'rotated_height_map.png'))
    plt.close()
    
    # Threshold the height map
    thresholded = threshold_height_map(height_map, min_height=0.3, max_height=0.7)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(thresholded, cmap='viridis')
    plt.colorbar(label='Height')
    plt.title('Thresholded Height Map (0.3 - 0.7)')
    plt.savefig(os.path.join(output_dir, 'thresholded_height_map.png'))
    plt.close()
    
    # Extract cross sections
    x_position = height_map.shape[0] // 2  # Middle row
    x_positions, x_heights = extract_cross_section(height_map, data, axis='x', position=x_position)
    
    plt.figure(figsize=(10, 6))
    plt.plot(x_positions, x_heights)
    plt.title(f'X Cross-Section at Row {x_position}')
    plt.xlabel('X Position')
    plt.ylabel('Height')
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'x_cross_section.png'))
    plt.close()
    
    # 5. Demonstrate export options
    print("\nDemonstrating export options...")
    
    # Export to NumPy formats
    npy_path = export_to_npy(height_map, os.path.join(output_dir, 'height_map.npy'))
    npz_path = export_to_npz(data, os.path.join(output_dir, 'tmd_data.npz'))
    txt_path = export_metadata_txt(data, os.path.join(output_dir, 'metadata.txt'))
    
    # Export image maps
    disp_path = os.path.join(output_dir, 'displacement_map.png')
    disp_img = convert_heightmap_to_displacement_map(height_map, disp_path)
    
    normal_path = os.path.join(output_dir, 'normal_map.png')
    normal_img = convert_heightmap_to_normal_map(height_map, normal_path)
    
    bump_path = os.path.join(output_dir, 'bump_map.png')
    bump_img = convert_heightmap_to_bump_map(height_map, bump_path)
    
    # Export 3D model
    stl_path = os.path.join(output_dir, 'height_map.stl')
    convert_heightmap_to_stl(
        height_map,
        filename=stl_path,
        x_length=data['x_length'],
        y_length=data['y_length'],
        z_scale=2.0  # Exaggerate height for better visualization
    )
    
    print("\nAll demonstrations complete! Output files are in the 'output' directory.")
    print("Here's a summary of the files created:")
    print(f"  - Raw height map visualization: {os.path.join(output_dir, 'raw_height_map.png')}")
    print(f"  - Processed height maps: cropped, flipped, rotated, thresholded")
    print(f"  - Cross-section: {os.path.join(output_dir, 'x_cross_section.png')}")
    print(f"  - Export formats:")
    print(f"    - NumPy: {npy_path}, {npz_path}")
    print(f"    - Metadata: {txt_path}")
    print(f"    - Image maps: displacement, normal, bump")
    print(f"    - 3D model: {stl_path}")


if __name__ == "__main__":
    main()

Generating synthetic TMD file: output\sample.tmd
Creating sample height map...

Processing TMD file...
Processing file: output\sample.tmd

File header (hex): 42696e61727920547275654d617020446174612046696c652076322e3000000000000000544d442044656d6f204578616d706c652046696c6500000000000000
File header (ASCII): Binary TrueMap Data File v2.0       TMD Demo Example File       
Error processing file: Could not find valid dimensions in the TMD file
Failed to process TMD file. Using synthetic height map directly.
Creating sample height map...

Demonstrating processing operations...


Traceback (most recent call last):
  File "c:\Users\antoi\OneDrive\Bureau\tmd\tmd\processor.py", line 69, in process
    metadata, height_map = process_tmd_file(self.file_path, debug=self.debug)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\antoi\OneDrive\Bureau\tmd\tmd\utils\utils.py", line 347, in process_tmd_file
    raise ValueError("Could not find valid dimensions in the TMD file")
ValueError: Could not find valid dimensions in the TMD file



Demonstrating export options...
Height map saved to NPY file: output\height_map.npy
TMD data saved to compressed NPZ file: output\tmd_data.npz
TMD metadata saved to text file: output\metadata.txt
Displacement map saved to output\displacement_map.png
Normal map saved to output\normal_map.png
Bump map saved to output\bump_map.png
ASCII STL file saved to output\height_map.stl

All demonstrations complete! Output files are in the 'output' directory.
Here's a summary of the files created:
  - Raw height map visualization: output\raw_height_map.png
  - Processed height maps: cropped, flipped, rotated, thresholded
  - Cross-section: output\x_cross_section.png
  - Export formats:
    - NumPy: output\height_map.npy, output\tmd_data.npz
    - Metadata: output\metadata.txt
    - Image maps: displacement, normal, bump
    - 3D model: output\height_map.stl


: 